Dataset :
Link : https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Library yang dibutuhkan

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

Inisialisasi dan Preprosesing Data

In [ ]:
train_dir = 'ClearDataset/train'
validation_dir = 'ClearDataset/valid'

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
class_names = list(train_generator.class_indices.keys())
print("Nama-nama kelas:", class_names)

Membangun Model CNN dengan Conv2D hingga 512

In [ ]:
# Membangun model CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Jumlah kelas sesuai dataset
])

Model setup early stoping

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,  # Menunggu 3 epoch jika akurasi tidak meningkat
    mode='max',
    verbose=1,
    restore_best_weights=True
)

Compile model

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stopping]
)

Analisis akurasi dengan plot diagram dan simpan model h5

In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Simpan model
model.save('Final_CekTandur.h5')

Uji Model H5 dengan data pada direktori

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf

# Path ke gambar yang akan diuji
test_image_path = '/content/kentang.jpeg'

# Memastikan file gambar ada
if not os.path.exists(test_image_path):
    raise FileNotFoundError(f"File gambar tidak ditemukan: {test_image_path}")

# Load model
model_path = 'Final_CekTandur.h5'
# Load model
model = tf.keras.models.load_model(model_path)

# Mapping kelas langsung dari class_indices
class_indices = {
    0: 'Anggur__Bercak_daun_isariopsis',
    1: 'Anggur__Esca(campak_hitam)',
    2: 'Anggur__Hitam_busuk',
    3: 'Anggur__Sehat',
    4: 'Apel__Busuk_hitam',
    5: 'Apel__Karat_apel_cedar',
    6: 'Apel__Keropeng_apel',
    7: 'Apel__Sehat',
    8: 'Jagung__Bercak_daun_abu-abu',
    9: 'Jagung__Busuk_daun',
    10: 'Jagung__Karat_umum',
    11: 'Jagung__Sehat',
    12: 'Kentang__Busuk_daun_dini',
    13: 'Kentang__Busuk_daun_telat',
    14: 'Kentang__Sehat',
    15: 'Tomat__Bercak_bakteri',
    16: 'Tomat__Bercak_daun',
    17: 'Tomat__Bercak_target',
    18: 'Tomat__Busuk_daun_dini',
    19: 'Tomat__Busuk_daun_telat',
    20: 'Tomat__Daun_keriting_kuning',
    21: 'Tomat__Jamur_septoria_lycopersici',
    22: 'Tomat__Sehat',
    23: 'Tomat__Tungau_laba-laba_Berbintik',
    24: 'Tomat__Virus_mosaik_tomat'
}

# Menyiapkan gambar untuk prediksi
def preprocess_image(image_path, target_size=(128, 128)):
    img = load_img(image_path, target_size=target_size)  # Load gambar
    img_array = img_to_array(img)  # Konversi ke array
    img_array = np.expand_dims(img_array, axis=0)  # Tambahkan dimensi batch
    img_array = img_array / 255.0  # Rescale
    return img_array

# Preprocess gambar
img_array = preprocess_image(test_image_path)

predicted_class = np.argmax(prediction, axis=1)[0] if prediction.ndim > 1 else np.argmax(prediction)
print(f"Prediksi kelas (indeks): {predicted_class}")

# Ambil label dari class_indices
predicted_label = class_indices[predicted_class]

# Tampilkan hasil
print(f"Gambar: {os.path.basename(test_image_path)}")
print(f"  Prediksi: {predicted_label}")
print(f"  Probabilitas: {prediction[0][predicted_class]:.2f}")
print("----")

Prediksi kelas (indeks): 1
Gambar: kentang.jpeg
  Prediksi: Anggur__Esca(campak_hitam)
  Probabilitas: 0.37
----
